In [6]:
import jax
import jax.numpy as jnp
from functools import partial

In [61]:
@jax.jit
def inner(x):
    print("hi from inner", x)
    return jnp.exp(x) + x

def outer(x):
    return inner(x) + inner(x.astype(int))

In [62]:
x = jnp.linspace(0, 1, 3)

In [63]:
jax.jit(outer)(x)

hi from inner Traced<ShapedArray(float32[3])>with<DynamicJaxprTrace(level=2/0)>
hi from inner Traced<ShapedArray(int32[3])>with<DynamicJaxprTrace(level=2/0)>


Array([2.       , 3.1487212, 7.4365635], dtype=float32)

In [54]:
jax.jit(outer)(x)

Array([ 4.7182817,  8.13041  , 13.107338 ], dtype=float32)

In [25]:
print(jax.jit(outer).lower(x).as_text())

hi from inner
module @jit_outer attributes {mhlo.num_partitions = 1 : i32, mhlo.num_replicas = 1 : i32} {
  func.func public @main(%arg0: tensor<3xf32>) -> (tensor<3xf32> {jax.result_info = ""}) {
    %0 = call @inner(%arg0) : (tensor<3xf32>) -> tensor<3xf32>
    %cst = stablehlo.constant dense<1.000000e+00> : tensor<f32>
    %1 = stablehlo.broadcast_in_dim %cst, dims = [] : (tensor<f32>) -> tensor<3xf32>
    %2 = stablehlo.add %arg0, %1 : tensor<3xf32>
    %3 = call @inner(%2) : (tensor<3xf32>) -> tensor<3xf32>
    %4 = stablehlo.add %0, %3 : tensor<3xf32>
    return %4 : tensor<3xf32>
  }
  func.func private @inner(%arg0: tensor<3xf32>) -> tensor<3xf32> {
    %0 = stablehlo.exponential %arg0 : tensor<3xf32>
    %1 = stablehlo.add %0, %arg0 : tensor<3xf32>
    return %1 : tensor<3xf32>
  }
}



In [22]:
print(jax.jit(outer).lower(x).as_text())

hi from inner
module @jit_outer attributes {mhlo.num_partitions = 1 : i32, mhlo.num_replicas = 1 : i32} {
  func.func public @main(%arg0: tensor<3xf32>) -> (tensor<3xf32> {jax.result_info = ""}) {
    %0 = stablehlo.exponential %arg0 : tensor<3xf32>
    %1 = stablehlo.add %0, %arg0 : tensor<3xf32>
    %cst = stablehlo.constant dense<1.000000e+00> : tensor<f32>
    %2 = stablehlo.broadcast_in_dim %cst, dims = [] : (tensor<f32>) -> tensor<3xf32>
    %3 = stablehlo.add %arg0, %2 : tensor<3xf32>
    %4 = stablehlo.exponential %3 : tensor<3xf32>
    %5 = stablehlo.add %4, %3 : tensor<3xf32>
    %6 = stablehlo.add %1, %5 : tensor<3xf32>
    return %6 : tensor<3xf32>
  }
}



In [19]:
print(jax.jit(outer).lower(x).compile().as_text())

hi from inner
hi from inner
HloModule jit_outer, is_scheduled=true, entry_computation_layout={(f32[3]{0})->f32[3]{0}}, allow_spmd_sharding_propagation_to_parameters={true}, allow_spmd_sharding_propagation_to_output={true}

%fused_computation (param_0.1: f32[3]) -> f32[3] {
  %param_0.1 = f32[3]{0} parameter(0)
  %exponential.1 = f32[3]{0} exponential(f32[3]{0} %param_0.1), metadata={op_name="jit(outer)/jit(main)/exp" source_file="/var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/1365869465.py" source_line=4}
  %add.3 = f32[3]{0} add(f32[3]{0} %exponential.1, f32[3]{0} %param_0.1), metadata={op_name="jit(outer)/jit(main)/add" source_file="/var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/1365869465.py" source_line=4}
  %constant.0 = f32[] constant(1)
  %broadcast.0 = f32[3]{0} broadcast(f32[] %constant.0), dimensions={}
  %add.2 = f32[3]{0} add(f32[3]{0} %param_0.1, f32[3]{0} %broadcast.0), metadata={op_name="jit(outer)/jit(main)/add" source_file="/var/fold

In [16]:
print(jax.jit(outer).lower(x).compile().as_text())

HloModule jit_outer, is_scheduled=true, entry_computation_layout={(f32[3]{0})->f32[3]{0}}, allow_spmd_sharding_propagation_to_parameters={true}, allow_spmd_sharding_propagation_to_output={true}

%fused_computation (param_0.1: f32[3]) -> f32[3] {
  %param_0.1 = f32[3]{0} parameter(0)
  %exponential.3 = f32[3]{0} exponential(f32[3]{0} %param_0.1), metadata={op_name="jit(outer)/jit(main)/jit(inner)/exp" source_file="/var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/3771274069.py" source_line=4}
  %add.5 = f32[3]{0} add(f32[3]{0} %exponential.3, f32[3]{0} %param_0.1), metadata={op_name="jit(outer)/jit(main)/jit(inner)/add" source_file="/var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/3771274069.py" source_line=4}
  %constant.0 = f32[] constant(1)
  %broadcast.0 = f32[3]{0} broadcast(f32[] %constant.0), dimensions={}
  %add.4 = f32[3]{0} add(f32[3]{0} %param_0.1, f32[3]{0} %broadcast.0), metadata={op_name="jit(outer)/jit(main)/add" source_file="/var/folders/rb

In [41]:
def fun(x):
    return jnp.linalg.cholesky(x)

x = jnp.eye(200)
x = x[None, :, :] + jnp.zeros((100, 200, 200))

jax.make_jaxpr(jax.vmap(jax.jit(fun)))(x)

{ lambda ; a:f32[100,200,200]. let
    b:f32[100,200,200] = pjit[
      name=fun
      jaxpr={ lambda ; c:f32[100,200,200]. let
          d:f32[100,200,200] = pjit[
            name=cholesky
            jaxpr={ lambda ; e:f32[100,200,200]. let
                f:f32[100,200,200] = transpose[permutation=(0, 2, 1)] e
                g:f32[100,200,200] = add e f
                h:f32[100,200,200] = div g 2.0
                i:f32[100,200,200] = transpose[permutation=(0, 2, 1)] h
                j:f32[100,200,200] = add h i
                k:f32[100,200,200] = div j 2.0
                l:f32[100,200,200] = cholesky k
                m:i32[200,200] = iota[dimension=0 dtype=int32 shape=(200, 200)] 
                n:i32[200,200] = add m 0
                o:i32[200,200] = iota[dimension=1 dtype=int32 shape=(200, 200)] 
                p:bool[200,200] = ge n o
                q:bool[100,200,200] = broadcast_in_dim[
                  broadcast_dimensions=(1, 2)
                  shape=(100, 200,

In [42]:
jax.make_jaxpr(jax.jit(jax.vmap(fun)))(x)

{ lambda ; a:f32[100,200,200]. let
    b:f32[100,200,200] = pjit[
      name=fun
      jaxpr={ lambda ; c:f32[100,200,200]. let
          d:f32[100,200,200] = pjit[
            name=cholesky
            jaxpr={ lambda ; e:f32[100,200,200]. let
                f:f32[100,200,200] = transpose[permutation=(0, 2, 1)] e
                g:f32[100,200,200] = add e f
                h:f32[100,200,200] = div g 2.0
                i:f32[100,200,200] = transpose[permutation=(0, 2, 1)] h
                j:f32[100,200,200] = add h i
                k:f32[100,200,200] = div j 2.0
                l:f32[100,200,200] = cholesky k
                m:i32[200,200] = iota[dimension=0 dtype=int32 shape=(200, 200)] 
                n:i32[200,200] = add m 0
                o:i32[200,200] = iota[dimension=1 dtype=int32 shape=(200, 200)] 
                p:bool[200,200] = ge n o
                q:bool[100,200,200] = broadcast_in_dim[
                  broadcast_dimensions=(1, 2)
                  shape=(100, 200,

In [43]:
f = jax.vmap(fun)
jax.block_until_ready(f(x))
%timeit jax.block_until_ready(f(x))

10.7 ms ± 66.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
f = jax.jit(jax.vmap(fun))
jax.block_until_ready(f(x))
%timeit jax.block_until_ready(f(x))

10.4 ms ± 20.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
def fun_vec(a, b):
    return a[None, :] + b

@partial(jax.vmap, in_axes=(None, 0))
def fun(a, b):
    return a + b

In [65]:
a = jnp.ones(5)
b = jnp.zeros((4, 5))

In [66]:
jax.make_jaxpr(fun_vec)(a, b)

{ lambda ; a:f32[5] b:f32[4,5]. let
    c:f32[1,5] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 5)] a
    d:f32[4,5] = add c b
  in (d,) }

In [67]:
jax.make_jaxpr(fun)(a, b)

{ lambda ; a:f32[5] b:f32[4,5]. let
    c:f32[1,5] = broadcast_in_dim[
      broadcast_dimensions=(np.int64(1),)
      shape=(1, 5)
    ] a
    d:f32[4,5] = add c b
  in (d,) }

In [74]:
@jax.jit
def fun(x):
    return jnp.exp(x) + x ** 1.234


# fun(x)

In [75]:
with jax.explain_cache_misses():
    fun(x)

TRACING CACHE MISS at /var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/1719737272.py:2 (<module>) because:
  never seen function:
    fun id=5132480816 defined at /var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/3464009699.py:1


In [76]:
with jax.explain_cache_misses():
    fun(x[:-1])

TRACING CACHE MISS at /var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/667903597.py:2 (<module>) because:
  never seen function:
    dynamic_slice id=5134251152 defined at /Users/danfm/jax/jax/_src/dispatch.py:98
TRACING CACHE MISS at /var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/667903597.py:2 (<module>) because:
  for fun defined at /var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/3464009699.py:1
  never seen input type signature:
    x: f32[2]
  closest seen input type signature has 1 mismatches, including:
    * at x, seen f32[3], but now given f32[2]


In [78]:
with jax.explain_cache_misses():
    fun(x.astype(int))

In [148]:
import numpy as np

# @jax.jit
def fun(x):
    return (np.eye(x.shape[0]) + np.ones((x.shape[0], x.shape[0]))) @ x

In [149]:
# fun(x)

In [150]:
# jax.make_jaxpr(fun)(x)

In [151]:
print(jax.jit(fun).lower(x).compile().as_text())

HloModule jit_fun, is_scheduled=true, entry_computation_layout={(f32[3]{0})->f32[3]{0}}, allow_spmd_sharding_propagation_to_parameters={true}, allow_spmd_sharding_propagation_to_output={true}

ENTRY %main.4 (Arg_0.1: f32[3]) -> f32[3] {
  %Arg_0.1 = f32[3]{0} parameter(0), metadata={op_name="x"}
  %constant.2 = f32[3,3]{1,0} constant({ { 2, 1, 1 }, { 1, 2, 1 }, { 1, 1, 2 } })
  ROOT %dot.3 = f32[3]{0} dot(f32[3,3]{1,0} %constant.2, f32[3]{0} %Arg_0.1), lhs_contracting_dims={1}, rhs_contracting_dims={0}, metadata={op_name="jit(fun)/jit(main)/dot_general" source_file="/var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/2797848076.py" source_line=5}
}




In [142]:
print(jax.jit(fun).lower(x).compile().as_text())

HloModule jit_fun, is_scheduled=true, entry_computation_layout={(f32[3]{0})->f32[3]{0}}, allow_spmd_sharding_propagation_to_parameters={true}, allow_spmd_sharding_propagation_to_output={true}

%fused_computation (param_0.1: f32[3]) -> f32[3] {
  %iota.3 = s32[3,3]{1,0} iota(), iota_dimension=0, metadata={op_name="jit(fun)/jit(main)/iota" source_file="/var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/266758819.py" source_line=5}
  %iota.2 = s32[3,3]{1,0} iota(), iota_dimension=1, metadata={op_name="jit(fun)/jit(main)/iota" source_file="/var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/266758819.py" source_line=5}
  %compare.0 = pred[3,3]{1,0} compare(s32[3,3]{1,0} %iota.3, s32[3,3]{1,0} %iota.2), direction=EQ, metadata={op_name="jit(fun)/jit(main)/eq" source_file="/var/folders/rb/0cbmk8n57sl2r9hlgzpl0wxm0181x6/T/ipykernel_51078/266758819.py" source_line=5}
  %convert.0 = f32[3,3]{1,0} convert(pred[3,3]{1,0} %compare.0), metadata={op_name="jit(fun)/jit(main)

In [106]:
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])